In [1]:
import tensorflow as tf

### Configuration
create_dataset = False

def in_google_colab():
    """Checks if the code is running in Google Colab

    Returns:
        bool: _description_
    """
    try:
        import google.colab
        return True
    except ImportError:
        return False

if in_google_colab():
    print("Running in Google Colab")
    # Install necessary packages in Google Colab
    !rm -r sample_data/
    !git clone https://github.com/naderinho/anesthesia_anomaly_detection
    !cp -r anesthesia_anomaly_detection/* .
    !rm -r anesthesia_anomaly_detection/
    !pip install vitaldb
    create_dataset = False
else:
    print("Running locally")

### Datasetpath
directory = 'data/'
datasetpath = 'dataset02/'
vitaldbpath = 'vitaldb_tiva/'

### Import the necessary libraries
import numpy as np
import pandas as pd
import vitaldb as vf
import matplotlib.pyplot as plt
import pickle

### Custom functions
import modules as md

Running locally


In [2]:
### Load the datasets
bis = md.VitalImport(directory= directory, dataset=datasetpath, vitalpath=vitaldbpath)
bis.load('00_bis.npz')

info = md.infoImport(directory= directory, dataset=datasetpath, vitalpath=vitaldbpath)
info.load('01_info.npz')

info_non_norm = md.infoImport(directory= directory, dataset=datasetpath, vitalpath=vitaldbpath)
info_non_norm.load('01_info_norm_none.npz')

bloodpressure = md.VitalImport(directory= directory, dataset=datasetpath, vitalpath=vitaldbpath)
bloodpressure.load('02_bloodpressure.npz')

etCO2 = md.VitalImport(directory= directory, dataset=datasetpath, vitalpath=vitaldbpath)
etCO2.load('02_etCO2.npz')

spO2 = md.VitalImport(directory= directory, dataset=datasetpath, vitalpath=vitaldbpath)
spO2.load('02_spO2.npz')

hr = md.VitalImport(directory= directory, dataset=datasetpath, vitalpath=vitaldbpath)
hr.load('02_hr.npz')

propofolrate = md.VitalImport(directory= directory, dataset=datasetpath, vitalpath=vitaldbpath)
propofolrate.load('03_propofol_rate.npz')

remifentanilrate = md.VitalImport(directory= directory, dataset=datasetpath, vitalpath=vitaldbpath)
remifentanilrate.load('03_remifentanil_rate.npz')

train_index, val_index, test_index = bis.split(np.array(bis.index))

# Seperate the Data in + 60 years and older patients

In [3]:
young_index_train = np.where(info_non_norm.train_dataset[:,1] <  60)
old_index_train   = np.where(info_non_norm.train_dataset[:,1] >= 60)

print(young_index_train[0].shape, old_index_train[0].shape)

young_index_val = np.where(info_non_norm.validation_dataset[:,1] <  60)
old_index_val   = np.where(info_non_norm.validation_dataset[:,1] >= 60)

print(young_index_val[0].shape, old_index_val[0].shape)

young_index_test = np.where(info_non_norm.test_dataset[:,1] <  60)
old_index_test   = np.where(info_non_norm.test_dataset[:,1] >= 60)

print(young_index_test[0].shape, old_index_test[0].shape)

(47,) (64,)
(10,) (10,)
(9,) (15,)


# Seperate the Data in Male and Female patients

In [4]:
m_index_train = np.where(info.train_dataset[:,0] == 0.5)
f_index_train = np.where(info.train_dataset[:,0] == -0.5)

print(m_index_train[0].shape, f_index_train[0].shape)

m_index_val = np.where(info.validation_dataset[:,0] == 0.5)
f_index_val = np.where(info.validation_dataset[:,0] == -0.5)

print(m_index_val[0].shape, f_index_val[0].shape)

m_index_test = np.where(info.test_dataset[:,0] == 0.5)
f_index_test = np.where(info.test_dataset[:,0] == -0.5)

print(m_index_test[0].shape, f_index_test[0].shape)

(74,) (37,)
(12,) (8,)
(18,) (6,)


# Load prediction data

In [5]:
# Load test prediction data
with open('prediction.pkl', 'rb') as f:
    y_pred = pickle.load(f)

### Predict on the test set
from utils.evaluation import phases_report, phases_report_std

# Testing Age

## > 60 years

In [6]:
phases_report(y_pred[old_index_test], bis.test_dataset[old_index_test], propofolrate.test_dataset[old_index_test])

,Prediction MSE,Baseline MSE,Prediction MAE,Baseline MAE,Prediction RMSE,Baseline RMSE
All,73.76,138.97,6.44,7.10,8.59,11.79
Induction,170.57,624.85,10.38,17.37,13.06,25.00
Maintenance,53.84,47.75,5.62,4.92,7.34,6.91
Recovery,125.36,374.76,8.51,13.62,11.20,19.36


## < 60 years

In [7]:
phases_report(y_pred[young_index_test], bis.test_dataset[young_index_test], propofolrate.test_dataset[young_index_test])

,Prediction MSE,Baseline MSE,Prediction MAE,Baseline MAE,Prediction RMSE,Baseline RMSE
All,92.96,141.25,7.60,8.23,9.64,11.88
Induction,189.70,660.01,11.34,20.05,13.77,25.69
Maintenance,80.66,79.34,7.13,6.76,8.98,8.91
Recovery,127.52,329.30,8.79,13.06,11.29,18.15


# Testing Sex

## Male

In [8]:
phases_report(y_pred[m_index_test], bis.test_dataset[m_index_test], propofolrate.test_dataset[m_index_test])

,Prediction MSE,Baseline MSE,Prediction MAE,Baseline MAE,Prediction RMSE,Baseline RMSE
All,72.75,122.47,6.51,6.94,8.53,11.07
Induction,174.45,600.22,10.78,17.60,13.21,24.50
Maintenance,58.87,53.44,5.93,5.27,7.67,7.31
Recovery,104.52,335.83,7.76,12.74,10.22,18.33


## Female

In [9]:
phases_report(y_pred[f_index_test], bis.test_dataset[f_index_test], propofolrate.test_dataset[f_index_test])

,Prediction MSE,Baseline MSE,Prediction MAE,Baseline MAE,Prediction RMSE,Baseline RMSE
All,115.31,208.12,8.44,9.94,10.74,14.43
Induction,186.70,750.27,10.57,20.57,13.66,27.39
Maintenance,91.49,93.06,7.64,7.47,9.56,9.65
Recovery,182.10,428.86,10.77,15.33,13.49,20.71
